In [1]:
# Matplotlib - 2D plotting library
from matplotlib import pyplot as plt
from matplotlib import rcParams

# Show matplotlib figures inline the notebook
%matplotlib inline

# Pandas - High-performance data analysis
import pandas

# Glob - Unix style pathname pattern expansion
# Used to aggregate folders and files into a single list so they can be iterated over
import glob

# Numpy - For handling of large, multi-dimensional arrays and matrices
import numpy as np
from numpy.linalg import solve

# Scipy - For accurate integration
from scipy.integrate import simps
from scipy.integrate import trapz
from scipy.optimize import curve_fit
import scipy.odr as sc

import csv

print 'Done'

Done


In [2]:
##########################################################################################
# Energy of incident (input) particles
InputEnergies = np.array([np.linspace(10,90, num=9, endpoint=True, dtype = int),
                          np.linspace(100,900, num=9, endpoint=True, dtype = int),
                          np.linspace(1100,1900, num=9, endpoint=True, dtype = int),
                          np.linspace(1000,9000, num=9, endpoint=True, dtype = int),
                          np.linspace(10000,90000, num=9, endpoint=True, dtype = int),
                          np.linspace(100000,900000, num=9, endpoint=True, dtype = int),
                            ]).flatten()

# Folder containing simulation data
DataFolder = '../G4AdEPTCubeSat_Results/Gammas/'

# Arrange the array into the proper order
# InputEnergies = np.delete(InputEnergies,18)
# InputEnergies = np.insert(InputEnergies, [9], [1000])
# InputEnergies = np.insert(InputEnergies, [45], [1000000])
InputEnergies = np.delete(InputEnergies,27)
InputEnergies = np.insert(InputEnergies, [18], [1000])
InputEnergies = np.insert(InputEnergies, [54], [1000000])
# ##########################################################################################
# Energy thresholds (in eV) for events that will be included in the analysis
Thr_Gas = 100.
print InputEnergies

[     10      20      30      40      50      60      70      80      90
     100     200     300     400     500     600     700     800     900
    1000    1100    1200    1300    1400    1500    1600    1700    1800
    1900    2000    3000    4000    5000    6000    7000    8000    9000
   10000   20000   30000   40000   50000   60000   70000   80000   90000
  100000  200000  300000  400000  500000  600000  700000  800000  900000
 1000000]


In [3]:
##########################################################################################
# Load the simulation data for each input energy
Data = []
n = 0
for InputEnergy in InputEnergies:
    # Create data file template from current energy 
    DataFilename = 'gamma_' + str(InputEnergy) + 'keV_Nr_100000000_ISO_nt_G4AdEPTCubeSat_t[0-7].csv'

    # Statistics variables for each input energy data
    nPos = 0
    nElec = 0
    nSubPos = 0
    nSubElec = 0
    nTotal = 0
    nSubThr = 0
    nCompton = 0
    nPair = 0
    nPhoto = 0
    nTriton = 0
    
    # Print energy 
    print '\n------------------------------------------------------------'
    print 'Energy: \t\t', str(InputEnergy), 'keV'

    ##########################################################################################
    # Process the simulation data files
    # Column 0 - Total energy deposited in the sensitive gas volume [eV]
    # Column 1 - Energy deposited by positrons in the sensitive gas volume [eV]
    # Column 2 - Energy deposited by positrons in the sensitive gas volume [eV]
    # Column 3 - Energy deposited by tritons in the sensitive gas volume [eV]
    # Column 4 - Track length of particles in the sensitive gas volume [mm]
    # Column 5 - Number of secondary electrons generated in the sensitive gas volume
    # Column 6 - Number of secondary photons generated in the sensitive gas volume
    # Column 7 - Number of secondary positrons generated in the sensitive gas volume
    # Column 8 - Number of secondary tritons generated in the sensitive gas volume
    for dataFile in glob.glob(DataFolder + DataFilename):
        # Readout CSV files
        try:
            dataFrame = pandas.read_csv(dataFile, usecols=[0,1,2,3,4,5,6,7,8], comment="#", error_bad_lines=False, skip_blank_lines=True).values

            # Determine the interaction type
            for i in range(1,len(dataFrame)):
                if dataFrame[i,0] > Thr_Gas:
    #                 if dataFrame[i,8] > 0:
    #                     nTriton += 1
                    if dataFrame[i,7] > 0 and dataFrame[i,5] > 0:
                        nPair += 1
                    else:
                        if dataFrame[i,6] > 0. and dataFrame[i,5] > 0.:
                            nCompton += 1
                        else:
                            nPhoto += 1

            # Increment statistics
            nPos += dataFrame[(dataFrame[:,1] > Thr_Gas),0].size
            nElec += dataFrame[(dataFrame[:,2] > Thr_Gas),0].size
            nSubPos += dataFrame[(dataFrame[:,1] <= Thr_Gas),0].size
            nSubElec += dataFrame[(dataFrame[:,2] <= Thr_Gas),0].size
            nTotal += dataFrame[(dataFrame[:,0] > Thr_Gas),0].size
            nSubThr += dataFrame[(dataFrame[:,0] <= Thr_Gas),0].size
            
        except:
            pass
        
    # Add statistics to data list (Convert to numpy array later)
    Data.append([InputEnergy,nPos,nElec,nSubPos,nSubElec,nTotal,nSubThr,nCompton,nPair,nPhoto,nTriton])
    
    # Print statistics
    print '------------------------------------------------------------'
    print 'Number of events above threshold: \t', nTotal
    print 'Number of events below threshold: \t', nSubThr
    print 'Number of Compton Scattering events: \t', nCompton
    print 'Number of Pair Production events: \t', nPair
    print 'Number of Photoelectric Effect events: \t', nPhoto
    print '------------------------------------------------------------'
    
# Convert statistics list to Numpy array
df = pandas.DataFrame(data=Data)
DataFolder2 = '../G4AdEPTCubeSat_Results/Gamma_Data/'
newFile = 'gamma_Data.csv'   

# Write the data to a CSV file
df.to_csv(DataFolder2+newFile)
print str(newFile) + " written to " + str(DataFolder2)

Data = np.asarray(Data)
print '\n'
print 'File writing completed'


------------------------------------------------------------
Energy: 		10 keV
------------------------------------------------------------
Number of events above threshold: 	0
Number of events below threshold: 	0
Number of Compton Scattering events: 	0
Number of Pair Production events: 	0
Number of Photoelectric Effect events: 	0
------------------------------------------------------------

------------------------------------------------------------
Energy: 		20 keV
------------------------------------------------------------
Number of events above threshold: 	242276
Number of events below threshold: 	325
Number of Compton Scattering events: 	398
Number of Pair Production events: 	0
Number of Photoelectric Effect events: 	241870
------------------------------------------------------------

------------------------------------------------------------
Energy: 		30 keV
------------------------------------------------------------
Number of events above threshold: 	469238
Number of events